#Importante!

# Monte seu drive ou suba diretamente o arquivo csv no ambiente

In [22]:
# from google.colab import drive
# drive.mount('/content/drive')

## <font color='blue'>Microsoft Power BI Para Business Intelligence e Data Science</font>

## <font color='blue'>Machine Learning com Linguagem Python e Power BI Dentro do Jupyter Notebook</font>

Problema de Negócio:

Considerando dados históricos de clientes que realizaram compras em nossa empresa, faça o agrupamento (segmentação) dos clientes por similaridade de características em 3 grupos e envie o relatório para a equipe de Marketing.

As etapas completas de limpeza, pré-processamento e transformação serão resumidas ou simplificadas nesse exemplo, pois o foco é obter os dados segmentados para aplicar no Power BI.

In [23]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.12.12


In [40]:
# Imports
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from google.colab import files

## Carregando os Dados

In [25]:
# Carrega os dados
df_dsa = pd.read_csv('/content/dados_clientes.csv')


In [26]:
type(df_dsa)

pandas.core.frame.DataFrame

In [27]:
# Visualiza as 10 primeiras linhas
df_dsa.head(10)

,id,idade,renda_anual,pontuacao_gastos
0,1,56,94740,90
1,2,69,136352,50
2,3,46,86617,62
3,4,32,114841,97
4,5,60,36896,51
5,6,25,145729,37
6,7,38,66175,96
7,8,56,27805,87
8,9,36,25237,78
9,10,40,135247,29


## Análise Exploratória

In [28]:
# Resumo estatístico
df_dsa[['idade', 'renda_anual', 'pontuacao_gastos']].describe()

,idade,renda_anual,pontuacao_gastos
count,500.000000,500.000000,500.000000
mean,44.732000,81557.166000,48.512000
std,15.239707,36764.379945,29.556946
min,18.000000,20384.000000,0.000000
25%,32.000000,49172.750000,24.000000
50%,45.000000,79219.000000,48.500000
75%,57.000000,113017.250000,73.250000
max,70.000000,149695.000000,100.000000


In [29]:
# Adicionei esse comando para obter informações nos dados

df_dsa.info()
df_dsa.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   id                500 non-null    int64
 1   idade             500 non-null    int64
 2   renda_anual       500 non-null    int64
 3   pontuacao_gastos  500 non-null    int64
dtypes: int64(4)
memory usage: 15.8 KB


,0
id,0
idade,0
renda_anual,0
pontuacao_gastos,0


## Pré-Processamento dos Dados

In [30]:
# Cria o padronizador dos dados
padronizador = StandardScaler()

##OBS: Em clusterização não é obrigatório dividir os dados em treino e teste, pois é uma técnica não supervisionada. Todavia, como boa prática, você analista pode fazer isso. Veja as etapas:

1. Seleção das colunas:
df_dsa_filtrado = df_dsa[['idade', 'renda_anual', 'pontuacao_gastos']]

2. Divisão em Treino e Teste (antes de qualquer padronização):

from sklearn.model_selection import train_test_split

dados_treino, dados_teste = train_test_split(df_dsa_filtrado, test_size=0.2, random_state=42)

3. Padronização correta
O fit_transform vai APENAS no treino:

treino_padronizado = padronizador.fit_transform(dados_treino)

print(treino_padronizado)

No teste, você usa APENAS o transform (sem o fit!):

teste_padronizado = padronizador.transform(dados_teste)

4. Usando modelo.

k = 3 (Vamos agrupar em 3 categorias)

kmeans = KMeans(n_clusters = k)

kmeans.fit(treino_padronizado)

dados_treino['cluster'] = kmeans.labels_

dados_treino.head(10)

5. Aplicando o modelo aos dados de teste que ficaram guardados

dados_teste['cluster'] = kmeans.predict(teste_padronizado)

##Nesse exemplo, não adotou-se a divisão em dados de treino e teste, portanto pode seguir a partir desse ponto do script

In [31]:
# Aplica o padronizador somente nas colunas de interesse
dados_padronizados = padronizador.fit_transform(df_dsa[['idade', 'renda_anual', 'pontuacao_gastos']])

In [32]:
# Visualiza os dados
print(dados_padronizados)

[[ 0.74012478  0.35893538  1.40506904]
 [ 1.59401387  1.49192537  0.05039391]
 [ 0.08328703  0.13776654  0.45679645]
 ...
 [-0.31081563  0.32661636  0.18586143]
 [-1.23038848 -1.48706069  1.43893592]
 [-1.03333716 -0.96992912 -0.59307677]]


## Construção do Modelo de Machine Learning Para Segmentação de Clientes

In [33]:
# Definimos o número de clusters (k)
k = 3

In [34]:
# Criamos o modelo K-means
kmeans = KMeans(n_clusters = k)

In [35]:
# Treinamento do modelo com os dados padronizados
kmeans.fit(dados_padronizados)

KMeans(n_clusters=3)

In [36]:
# Atribuímos os rótulos dos clusters aos clientes
df_dsa['cluster'] = kmeans.labels_

In [37]:
# Exibe o resultado (10 primeiras linhas)
df_dsa.head(10)

,id,idade,renda_anual,pontuacao_gastos,cluster
0,1,56,94740,90,2
1,2,69,136352,50,1
2,3,46,86617,62,2
3,4,32,114841,97,2
4,5,60,36896,51,0
5,6,25,145729,37,1
6,7,38,66175,96,2
7,8,56,27805,87,2
8,9,36,25237,78,2
9,10,40,135247,29,1


In [38]:
df_dsa['cluster'].value_counts()

,count
cluster,
2,182
1,166
0,152


In [39]:
# Salvamos o resultado em disco

df_dsa.to_csv('segmentos.csv', index=False)
files.download('segmentos.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Fim